# 7 Solution Methods to Solve the Growth Model with Python

This notebook is part of a computational appendix that accompanies the paper

> MATLAB, Python, Julia: What to Choose in Economics?
> > Coleman, Lyon, Maliar, and Maliar (2017)

In order to run the codes in this notebook you will need to install and configure a few Python packages. We recommend downloading [Anaconda](https://www.continuum.io/downloads) and/or following the instructions on [quantecon.org](https://lectures.quantecon.org/py/getting_started.html). Once your Python installation is up and running, there are a few additional packages you will need in order to run the code here. To do this, you should remove the `# ` in front of the following commands and run the block below.

In [443]:
# ! pip install git+https://github.com/EconForge/interpolation.py.git
# ! pip install quantecon
# ! pip install csaps

In [444]:
import numpy as np
import scipy.linalg as la
import scipy.optimize as opt
import time
import quantecon as qe
import matplotlib.pyplot as plt

from collections import namedtuple
from scipy.interpolate import interp1d, UnivariateSpline
from scipy.misc import derivative
from interpolation.complete_poly import (CompletePolynomial,
                                         n_complete, complete_polynomial,
                                         complete_polynomial_der,
                                         _complete_poly_impl,
                                         _complete_poly_impl_vec,
                                         _complete_poly_der_impl,
                                         _complete_poly_der_impl_vec)
from numba import jit, vectorize

## Model

This section gives a short description of the commonly used stochastic Neoclassical growth model.

There is a single infinitely-lived representative agent who consumes and saves using capital. The consumer discounts the future with factor $\beta$ and derives utility from only consumption. Additionally, saved capital will depreciate at $\delta$.

The consumer has access to a Cobb-Douglas technology which uses capital saved from the previous period to produce and is subject to stochastic productivity shocks.

Productivity shocks follow an AR(1) in logs.

The agent's problem can be written recursively using the following Bellman equation


$$
\begin{align}
  V(k_t, z_t) &= \max_{k_{t+1}} u(c_t) + \beta E \left[ V(k_{t+1}, z_{t+1}) \right] \\
  &\text{subject to } \\
  c_t &= z_t f(k_t) + (1 - \delta) k_t - k_{t+1} \\
  \log z_{t+1} &= \rho \log z_t + \sigma \varepsilon
\end{align}
$$

$$
\begin{align*}
    V(X, z) &= \max_{\tilde{c},\tilde{s}} \biggl\{0,  u(c) + \beta E \bigl[V(X', z') \bigr] \biggr\}\\
    & \text{ s.t. } \tilde{c} \geq 0; \qquad \tilde{s} \geq 0 \\
    & \phantom{\text{ s.t. } }N\tilde{c} + N\tilde{s} \leq X \\
    & \phantom{\text{ s.t. } } X' = \max \biggl\{0, \bigl[X - N\tilde{c}  - N\tilde{s} - z \bigl(\frac{\tilde{c}}{\tilde{s}}\bigr)^\phi \bigr](1 + g_x) \biggr\} \\
    & \phantom{\text{ s.t. } } u(\tilde{c}) = \bar{u} + \ln(\tilde{c}) \\
    & \phantom{\text{ s.t. } } z \sim Exp( \bar{z} ) 
\end{align*} 
$$

## Python Code

We begin by defining a `namedtuple` that contains the parameters of our model. This is useful to pass the parameters around to functions that are just-in-time (JIT) compiled by `numba`.

In [445]:
#
# Create a named tuple type that we can pass into the jitted functions
# so that we don't have to pass parameters one by one
#
Params = namedtuple("Params", ["N", "beta", "g", "a_bar", "eta", "u_bar"])

@jit(nopython=True)
def param_unpack(params):
    "Unpack parameters from the Params type"
    out = (params.N, params.beta, params.g,
           params.a_bar, params.eta, params.u_bar)

    return out

We will then define various helper functions to ensure that we [don't repeat ourselves](https://lectures.quantecon.org/py/writing_good_code.html#don-t-repeat-yourself) and that the inner functions can be JIT compiled.

In [446]:
#
# Helper functions to make sure things are jitted
#
@vectorize(nopython=True)
def u(c, eta, u_bar):
    "log utility function"
    return -1e10 if c < 1e-10 else (u_bar + np.log(c)) if eta==1 else (u_bar + (c**(1-eta)/(1-eta)))

@vectorize(nopython=True)
def du(c,eta):
    "Derivative of log utility function"
    return 1e10 if c < 1e-10 else c**(-eta)

@vectorize(nopython=True)
def duinv(u,eta):
    "Inverse of the derivative of the CRRA utility function"
    return u**(-1.0/eta )

@vectorize(nopython=True)
def f(X):
    "production function"
    return X

@vectorize(nopython=True)
def df(X):
    "Derivative of production function"
    return 1

@vectorize(nopython=True)
def discount_factor(beta, a_bar, c, w):
    "risk adjusted discount factor"
    return beta*a_bar*w/(c + a_bar*w)

@vectorize(nopython=True)
def next_x(X, N, g, c, w):
    # next period's CPR given c and w
    return (X - N*(c + w))*(1+g)

@vectorize(nopython=True)
def vector_dampen(vec1, vec2, dampen=1):
    return dampen*vec1 + (1-dampen)*vec2

@vectorize(nopython=True)
def expendables(X, Xp, N, g):
    # get expendables given X and X prime
    return (1/N)*(X - Xp/(1 + g))

@vectorize(nopython=True)
def logu_c_from_exp(Vp, E, beta, a):
    # get expendables given X and X prime
    return -(2*E*a - 2*E*a**2 + E*a*(Vp*beta*(4*a + Vp*beta - 4))**0.5 - E*Vp*a*beta)/(2*(a**2 - 2*a + 1))

def c_curve(x, a, b, c):
    return a*np.log(x) + b*x + c

def w_curve(x, a, b, c):
    return a*np.log(x) + b*x + c

def v_curve(x, a, b, c, d):
    return np.nan_to_num(
        a*x + b*x**(c)
    , nan=0)

@vectorize(nopython=True)
def trunc_to_one(X):
    return X if X >= 1 else 1

@vectorize(nopython=True)
def psurvival(a_bar, c, w):
    return a_bar*w/(c + a_bar*w)


This code block contains other functions that are useful for computing the policy using the envelope condition (to be discussed later), simulating, and computing Euler errors.

In [447]:
@jit(nopython=True)
def env_cond_kp(temp, params, degree, v_coeffs, Xt):
    # Unpack parameters
    N, beta, g, a_bar, eta, u_bar = param_unpack(params)

    # Compute derivative of VF wrt k
    _complete_poly_der_impl_vec(np.array([Xt, zt]), degree, 0, temp)

    c = duinv(np.dot(temp, v_coeffs) / (1/N))

    return expendables_t(Xt) - c


@jit(nopython=True)
def jit_simulate_ncgm(params, degree, v_coeffs, T, nburn, shocks):
    "Simulates economy using envelope condition as policy rule"
    # Unpack parameters
    N, beta, g, a_bar, eta, u_bar = param_unpack(params)

    # Allocate space for output
    xsim = np.empty(T+nburn)
    zsim = np.empty(T+nburn)
    xsim[0], zsim[0] = 1.0, 1.0 # may need to modify initial values

    # Allocate space for temporary vector to fill with complete polynomials
    temp = np.empty(n_complete(2, degree))

    # Simulate
    for t in range(1, T+nburn):
        # Evaluate policy for today given yesterdays state
        xp = env_cond_kp(temp, params, degree, v_coeffs, xsim[t-1], zsim[t-1])
        
        # Draw new z and update k using policy from above
        zsim[t] = shocks[t]
        xsim[t] = kp

    return xsim[nburn:], zsim[nburn:]


@jit(nopython=True)
def jit_ee(params, degree, v_coeffs, nodes, weights, xs, zs):
    # Unpack parameters
    N, beta, g, a_bar, eta, u_bar = param_unpack(params)

    # Allocate space for temporary vector to fill with complete polynomials
    temp = np.empty(n_complete(2, degree))
    T = xs.size
    Qn = weights.size

    # Allocate space to store euler errors
    ee = np.empty(T)

    # Iterate over all ks and zs
    for t in range(T):
        # Current states
        x, z = xs[t], zs[t]

        # Compute decision for kp and implied c
        x1 = env_cond_kp(temp, params, degree, v_coeffs, x, z)
        c = expendables_t(k, z, A, alpha, delta) - k1

        # Compute euler error for period t
        lhs = du(c, gamma)
        rhs = 0.0
        for i in range(Qn):
            # Get productivity tomorrow
            z1 = z**rho * np.exp(nodes[i])

            # Compute decision for kpp and implied c
            k2 = env_cond_kp(temp, params, degree, v_coeffs, k1, z1)
            c1 = expendables_t(k1, z1, A, alpha, delta) - k2
            rhs = rhs + weights[i]*du(c1, gamma)*(1-delta+df(k1, z1, A, alpha))

        ee[t] = np.abs(1.0 - beta*rhs/lhs)

    return ee


We also now define a class that contains

1. Parameters of the growth model
2. Grids used for approximating the solution
3. Nodes and weights used to approximate integration

This again helps us maintain all of the relevant information in one place and simplifies passing it to other functions.

In [448]:
class Model(object):
    """
    The stochastic Neoclassical Growth model contains
    parameters which include

    * alpha: Capital share in output
    * beta: discount factor
    * delta: depreciation rate
    * gamma: risk aversion
    * rho: persistence of the log productivity level
    * sigma: standard deviation of shocks to log productivity
    """
    def __init__(self, N=1, beta=0.9, g_x=0.005, g_a=0.015, a_bar=1000, 
                 eta=1, u_bar=0, 
                 xmin=1, xmax=2000, nx=25):
        # calculate g from g_a and g_x 
        g = g_a + g_x + g_a*g_x

        # Household parameters
        self.beta, self.eta, self.u_bar, = beta, eta, u_bar

        # Firm/technology parameters
        self.N, self.g, self.a_bar = N, g, a_bar

        # Create t grids
        self.xgrid = N*np.linspace(xmin, xmax, nx)
        self.grid = np.reshape(self.xgrid, [self.xgrid.shape[0], 1])
        
        self.ns = nx
        
    def _unpack_params(self):
        out = (self.N, self.beta, self.g, 
               self.a_bar, self.eta, self.u_bar)
        return out

    def _unpack_grids(self):
        out = (self.xgrid, self.grid)
        return out


## Solution Methods

In this notebook, we describe the following solution methods:

* Conventional Value Function Iteration
* Envelope Condition Value Function Iteration
* Envelope Condition Derivative Value Function Iteration
* Endogenous Grid Value Function Iteration
* Conventional Policy Function Iteration
* Envelope Condition Policy Function Iteration
* Euler Equation Method

Each of these solution methods will have a very similar structure that follows a few basic steps:

1. Guess a function (either value function or policy function).
2. Using this function, update our guess of both the value and policy functions.
3. Check whether the function we guessed and what it was updated to are similar enough. If so, proceed. If not, return to step 2 using the updated functions.
4. Output the policy and value functions.

In order to reduce the amount of repeated code and keep the exposition as clean as possible (the notebook is plenty long as is...), we will define a class for each solution method that inherit various properties from a general solution parent class. The parent class will contain methods which apply to all of the other solution methods such as a general solve method, computing expectations, simulating, etc... This implementation may feel strange at first if one isn't familiar with object oriented programming, but these concepts can be powerful when properly used.

In [449]:
class GeneralSolution:
    """
    This is a general solution method. We define this, so that we can
    sub-class it and define specific update methods for each particular
    solution method
    """
    def __init__(self, ncgm, dx=1e-6, maxfev=10000, prev_sol=None):
        self.Xinf = None
        # Save model and approximation degree
        self.ncgm, self.dx, self.maxfev = ncgm, dx, maxfev

        # Unpack some info from ncgm
        N, beta, g, a_bar, eta, u_bar = self._unpack_params()
        X = self.ncgm.xgrid

        # Use parameter values from model to create a namedtuple with
        # parameters saved inside
        self.params = Params(N, beta, g, a_bar, eta, u_bar)

        # Update to fill initial value and policy matrices
        # If we give it another solution type then use it to
        # generate values and policies
        if issubclass(type(prev_sol), GeneralSolution):
            self.CP = c_curve(X, *prev_sol.c_params)
            self.WP = w_curve(X, *prev_sol.w_params)
            self.VF = v_curve(X, *prev_sol.v_params)
        # If we give it a tuple then assume it is (policy, value) pair
        elif type(prev_sol) is tuple:
            self.CP = prev_sol[0]
            self.WP = prev_sol[1]
            self.VF = prev_sol[2]
        # Otherwise guess a constant value function and a policy
        # of roughly steady state
        else:
            # guess VF
            self.VF = X # np.ones(ncgm.ns)
            
            # make a guess
            self.CP = X/(4*N)
            self.WP = X/(4*N)


        # Params based on guesses
        self.c_params, _ = opt.curve_fit(c_curve, X, self.CP, maxfev=self.maxfev)
        self.w_params, _ = opt.curve_fit(w_curve, X, self.WP, maxfev=self.maxfev)
        self.v_params, _ = opt.curve_fit(v_curve, X, self.VF, maxfev=self.maxfev)

    def _unpack_params(self):
        return self.ncgm._unpack_params()
    
    def build_all(self, X=None):
        if X is None:
            X = self.ncgm.xgrid
            
        CP = self.build_CP(X)
        WP = self.build_WP(X)
        VF = self.build_VF(X)
        
        # corner solution
        CP_corner = X/self.ncgm.N
        WP_corner = X*0 
        VF_corner = u(CP_corner, self.ncgm.eta, self.ncgm.u_bar)

        corner_idx = (CP > CP_corner) + (CP < 0) + (WP > CP_corner) + (WP < 0) + (VF < VF_corner)
        
        CP[corner_idx] = CP_corner[corner_idx]
        WP[corner_idx] = WP_corner[corner_idx]
        VF[corner_idx] = VF_corner[corner_idx]  
        
        return CP, WP, VF
    
    def build_all_scalar(self, x): 
        cp = c_curve(x, *self.c_params)
        wp = w_curve(x, *self.w_params)
        vf = v_curve(x, *self.v_params)
        
        # corner solution
        cp_corner = x/self.ncgm.N
        wp_corner = 0
        vf_corner = u(cp_corner, self.ncgm.eta, self.ncgm.u_bar)
        
        if (cp > cp_corner) or (cp < 0) or (wp > cp_corner) or (wp < 0) or (vf < vf_corner):
                return cp_corner, wp_corner, vf_corner
            
        return cp, wp, vf

        
    def build_VF(self, X=None):
        """
        Using the current coefficients, this builds the value function
        for all states
        """
        if X is None:
            X = self.ncgm.xgrid
            
        VF = v_curve(X, *self.v_params)

        return VF

    def build_CP(self, X=None):
        """
        Using the current coefficients, this builds the policy function
        for all states
        """
        if X is None:
            X = self.ncgm.xgrid
            
        CP = np.clip(c_curve(X, *self.c_params), 0, X/self.ncgm.N)

        return CP
    
    def build_WP(self, X=None):
        """
        Using the current coefficients, this builds the policy function
        for all states
        """
        if X is None:
            X = self.ncgm.xgrid
            
        WP = np.clip(w_curve(X, *self.w_params), 0, X/self.ncgm.N)

        return WP

    def update_v(self, new_v_params, dampen=1):
        """
        Updates the coefficients for the value function
        """
#         self.v_coeffs = (1-dampen)*self.v_coeffs + dampen*new_v_coeffs
        self.v_params = (1-dampen)*self.v_params + dampen*new_v_params

        return None

    def update_c(self, new_c_params, dampen=1):
        """
        Updates the coefficients for the policy function
        """
#         self.c_coeffs = (1-dampen)*self.c_coeffs + dampen*new_c_coeffs
        self.c_params = (1-dampen)*self.c_params + dampen*new_c_params

        return None
    
    def update_w(self, new_w_params, dampen=1):
        """
        Updates the coefficients for the policy function
        """
#         self.w_coeffs = (1-dampen)*self.w_coeffs + dampen*new_w_coeffs
        self.w_params = (1-dampen)*self.w_params + dampen*new_w_params

        return None

    def update(self):
        """
        Given the current state of everything in solution, update the
        value and policy coefficients
        """
        emsg = "The update method is implemented in solution specific classes"
        emsg += "\nand cannot be called from `GeneralSolution`"
        raise ValueError(emsg)

    
    def compute_params(self, cp, wp, VF):
        new_c_params, _ = opt.curve_fit(c_curve, self.ncgm.xgrid, cp, maxfev=self.maxfev)
        new_w_params, _ = opt.curve_fit(w_curve, self.ncgm.xgrid, wp, maxfev=self.maxfev)
        new_v_params, _ = opt.curve_fit(v_curve, self.ncgm.xgrid, VF, maxfev=self.maxfev) 
        
        return new_c_params, new_w_params, new_v_params

    def compute_EV_scalar(self, Xn):
        val = v_curve(trunc_to_one(Xn), *self.v_params)
        
        return val

    def compute_dEV_scalar(self, Xn):
        v_curve_curried = lambda x: v_curve(x, *self.v_params)
        val = derivative( v_curve_curried, trunc_to_one(Xn), self.dx)
        
        return val 

    def compute_EV(self, Xn=None):
        """
        Compute the expected value
        """

        # Use policy to compute cp and wp
        if Xn is None:
            # Unpack parameters
            N, beta, g, a_bar, eta, u_bar = self._unpack_params()
            X = self.ncgm.xgrid
            cp = c_curve(X, *self.c_params)
            wp = w_curve(X, *self.w_params)
            Xn = next_x(X, N, g, cp, wp)

        EV = v_curve(trunc_to_one(Xn), *self.v_params)
        
        return EV


    def compute_dEV(self, Xn=None):
        """
        Compute the derivative of the expected value
        """
        # Use policy to compute cp and wp
        if Xn is None:
            # Unpack parameters
            N, beta, g, a_bar, eta, u_bar = self._unpack_params()
            X = self.ncgm.xgrid
            cp = c_curve(X, *self.c_params)
            wp = w_curve(X, *self.w_params)
            Xn = next_x(X, N, g, cp, wp)
        
        v_curve_curried = lambda x: v_curve(x, *self.v_params)
        val = derivative( v_curve_curried, trunc_to_one(Xn), self.dx)
        
        return dEV


    def compute_distance(self, cp, wp, VF):
        """
        Computes average distance between policy functions
        """
        return np.max([
            np.max(np.abs(1.0 - (cp+1e-12)/(self.CP+1e-12))), 
            np.max(np.abs(1.0 - (wp+1e-12)/(self.WP+1e-12)))
        ])
    

    def solve(self, tol=1e-3, maxiter=2500, verbose=False, nskipprint=25):
        # Iterate until convergence
        dist, it = 10.0, 0
        while (dist>tol) and (it<maxiter):
            # Run solution specific update code
            cp, wp, VF = self.update()

            # Compute new policy and value coeffs
            new_c_params, new_w_params, new_v_params = self.compute_params(cp, wp, VF)

            # Update distance and iterations
            dist = self.compute_distance(cp, wp, VF)
            self.CP, self.WP, self.VF = cp, wp, VF
            it += 1
            if verbose and it%nskipprint == 0:
                print(it, dist)

            # Update all coefficients
            self.update_c(new_c_params)
            self.update_w(new_w_params)
            self.update_v(new_v_params)

        # After finishing iteration, iterate to convergence using policy
        if not isinstance(self, IterateOnPolicy):
            sol_iop = IterateOnPolicy(self.ncgm, self.dx, self.maxfev, self)
            cp, wp, VF = sol_iop.solve(tol=1e-3)

            # Save final versions of everything
            self.CP, self.WP, self.VF = cp, wp, VF
            
            new_c_params, new_w_params, new_v_params = sol_iop.compute_params(cp, wp, VF)
            self.update_c(new_c_params)
            self.update_w(new_w_params)
            self.update_v(new_v_params)

        return self.CP, self.WP, self.VF
    
    def solve_full(self, tol=1e-3, maxiter=2500, verbose=False, nskipprint=25):
        N, beta, g, a_bar, eta, u_bar = self._unpack_params()
        
        # solve as infinite horizon and store results
        self.CP_inf, self.WP_inf, self.VF_inf = self.solve(tol, maxiter, verbose, nskipprint)
        
        self.CP, self.WP, self.VF = self.build_all()
        
        return self.CP, self.WP, self.VF
        

    def simulate(self, X=None, T=1000, s=1):
        """
        Simulates the neoclassical growth model with policy function
        given by self.KP. Simulates for `T` periods and discarsd first
        nburn `observations`
        """
        N, beta, g, a_bar, eta, u_bar = self._unpack_params()
        # X is starting point, if None, set to N*274
        if X is None:
            X = N*274
            
        def recurse_bellman(x, t):
            if x <= 0:
                return 0, 0, x, t
            if t >= T:
                return u(x/N, eta, u_bar), 1, x, T
            
            c_corner = x/N 
            u_corner = u(c_corner, eta, u_bar)
            
            c = s*self.build_CP(x)
            w = s*self.build_WP(x)

            
            if w <= 0:
                c = c_corner 
                
            m_t = psurvival(a_bar, c, w)
            u_t = u(c, eta, u_bar)
            x_prime = next_x(x, N, g, c, w)
            
            v_prime, m_prime, final_state, nperiods = recurse_bellman(x_prime, t+1)
            
            if u_t + v_prime < u_corner:
                return u_corner, 0, 0, t+1
            
            return u_t + v_prime, m_t*m_prime, final_state, nperiods
        
        return recurse_bellman(X, 0)
        

    def ee_residuals(self, ksim=None, zsim=None, Qn=10, seed=42):
        """
        Computes the Euler Equation residuals of a simulated capital and
        productivity levels (ksim and zsim) and uses Qn nodes for computing
        the expectation
        """
        if (ksim is None) or (zsim is None):
            ksim, zsim = self.simulate(T=10000, nburn=200, seed=seed)

        nodes, weights = qe.quad.qnwnorm(Qn, 0.0, self.ncgm.sigma**2)
        ee = jit_ee(self.params, self.degree, self.v_coeffs,
                    nodes, weights, ksim, zsim)

        return np.log10(np.mean(ee)), np.log10(np.max(ee))
    
    def compute_Xdot(self, X):
        N, beta, g, a_bar, eta, u_bar = self._unpack_params()
        cp = c_curve(X, *self.c_params)
        wp = w_curve(X, *self.w_params)
        Xn = next_x(X, N, g, cp, wp)
        
        return Xn - X
        
    def compute_Xinflection(self):
        N, beta, g, a_bar, eta, u_bar = self._unpack_params()
        floor = N
        ceil = N*1000
        for i in range(4):
            X = np.linspace(floor, ceil, 100)
            Xdot = self.compute_Xdot(X)
            idx = np.argmin(Xdot[Xdot > 0])
            floor = X[idx]
            if idx < 99:
                ceil = X[idx + 1]
        
        return floor

### Iterating to Convergence (given policy)

This isn't one of the methods described above, but it is used as an element of a few of our methods (and also as a way to get a first guess at the value function). This method takes an initial policy function, $\bar{k}(k_t, z_t)$, as given, and then, without changing the policy, iterates until the value function has converged.

Thus the "update section" of the algorithm in this instance would be:

* Leave policy function unchanged
* At each point of grid, $(k_t, z_t)$, compute $\hat{V}(k_t, z_t) = u(c(\bar{k}(k_t, z_t))) + \beta E \left[ V(\bar{k}(k_t, z_t), z_{t+1}) \right]$

We override two of the methods from `GeneralSolution`

* `compute_distance` because when we are iterating to convergence on the value function we want to check distnace using value function rather than policy function
* `compute_coefficients` because we don't need to update the policy functions coefficients.

The `update` method just repeatedly applies a particular policy function to update the value function.

In [450]:
class IterateOnPolicy(GeneralSolution):
    """
    Subclass of the general solution method. The update method for this
    class simply computes the fixed point of the value function given
    a specific policy
    """
    def compute_distance(self, cp, wp, VF):
        """
        Computes distance between policy functions. When we are
        iterating on a specific policy, we would like to compute
        distances by the difference between VFs
        """
        return np.max(np.abs(1.0 - (VF )/(self.VF )))

    def compute_coefficients(self, cp, wp, VF):
        """
        Given a policy and value return corresponding coefficients.
        When we are iterating on a specific policy, we don't want to
        update the policy coefficients.
        """
        new_v_coeffs = la.lstsq(self.Phi, VF)[0]

        return self.c_coeffs, self.w_coeffs, new_v_coeffs
    
    def compute_params(self, cp, wp, VF):
        new_v_params, _ = opt.curve_fit(v_curve, self.ncgm.xgrid, VF, maxfev=self.maxfev) 
        
        return self.c_params, self.w_params, new_v_params
    
    def update(self):
        # Unpack parameters
        N, beta, g, a_bar, eta, u_bar = self._unpack_params()
        X = self.ncgm.xgrid
        
        Xn = next_x(X, N, g, self.CP, self.WP)

        # Update the value function
        VF = u(self.CP, eta, u_bar) + beta*np.maximum(self.compute_EV(Xn), np.zeros(self.ncgm.ns))

        return self.CP, self.WP, VF

### Conventional Value Function Iteration

This is one of the first solution methods for macroeconomics a graduate student in economics typically learns.

In this solution method, one takes as given a value function, $V(k_t, z_t)$, and then solves for the optimal policy given the value function.

The update section takes the form:

* For each point, $(k_t, z_t)$, numerically solve for $c^*(k_t, z_t)$ to satisfy the first order condition $u'(c^*) = \beta E \left[ V_1((1 - \delta) k_t + z_t f(k_t) - c^*, z_{t+1}) \right]$
* Define $k^*(k_t, z_t) = (1 - \delta) k_t + z_t f(k_t) - c^*(k_t, z_t)$
* Update value function according to $\hat{V}(k_t, z_t) = u(c^*(k_t, z_t)) + \beta E \left[ V(k^*(k_t, z_t), z_{t+1}) \right]$


* For each point, $(X_t, z_t)$, numerically solve for $c^*(X_t, z_t)$ to satisfy the first order condition $u'(\tilde{c}^\ast) = \beta E \left[ V_1((1 - \delta) k_t + z_t f(k_t) - c^*, z_{t+1}) \right]$
* Define $k^*(k_t, z_t) = (1 - \delta) k_t + z_t f(k_t) - c^*(k_t, z_t)$
* Update value function according to $\hat{V}(k_t, z_t) = u(c^*(k_t, z_t)) + \beta E \left[ V(k^*(k_t, z_t), z_{t+1}) \right]$




In [451]:
class VFI(GeneralSolution):
    """
    Updates the coefficients and value functions using the VFI
    method
    """
    def update(self):
        """
        Updates the coefficients and value functions using the VFI_ECM
        method
        """
        # Unpack parameters
        N, beta, g, a_bar, eta, u_bar = self._unpack_params()
        xgrid = self.ncgm.xgrid
        n_state = xgrid.shape[0]

        # Get the policy and update it
        cp = np.empty(n_state)
        wp = np.empty(n_state)
        VF = np.empty(n_state)
        for i_s in range(n_state):
            # Pull out current vals
            X = xgrid[i_s]
            def _f(xp):
                EV = self.compute_EV_scalar(xp)
                Exp = expendables(X, xp, N, g)
                _cp = np.clip(
                        np.nan_to_num(logu_c_from_exp(EV, Exp, beta, a_bar), nan=X/N),
                        0, X/N
                )
                _wp = Exp - _cp
                return -(u(_cp, 1, u_bar) + discount_factor(beta, a_bar, _cp, _wp)*EV)**2
                
            
            _xp = opt.fminbound(_f, 0, X*(1+g), xtol=1e-12)
            EV = self.compute_EV_scalar(_xp)
            Exp = expendables(X, _xp, N, g)
            _cp = np.clip(
                        np.nan_to_num(logu_c_from_exp(EV, Exp, beta, a_bar), nan=X/N),
                        0, X/N
                )
            _wp = Exp - _cp
            _vf = u(_cp, 1, u_bar) + discount_factor(beta, a_bar, _cp, _wp)*EV
            
            
            if _vf < 0:
                cp[i_s] = X/N
                wp[i_s] = 0
                VF[i_s] = u(X/N, eta, u_bar)
            else:                
                cp[i_s] = _cp
                wp[i_s] = _wp
                VF[i_s] = _vf

        return cp, wp, VF


## A Horse Race

We can now run a horse race to compare the methods in terms of both accuracy and speed.

In [452]:
ncgm = Model(N=1, a_bar=1000, xmin=10, xmax=1000, nx=100)

# First guess
vp = IterateOnPolicy(ncgm, maxfev=50000)
vp.solve(tol=1e-9)

np.random.seed(61089)


print('vp done')
dx=1e-6
new_sol = vp
new_sol = VFI(ncgm, dx=dx, maxfev=50000, prev_sol=new_sol)
ts = time.time()
new_sol.solve(tol=1e-5, verbose=True, nskipprint=25)
time_took = time.time() - ts

/Users/mikemahon/anaconda3/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/Users/mikemahon/anaconda3/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


vp done


/Users/mikemahon/anaconda3/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


25 0.9261520835626189
50 0.18500058879003367
75 0.19132947978156378
100 0.7755018394557562
125 0.12161564461422947
150 0.19127560023431034
175 0.22032070990378871
200 0.1450244211088283
225 0.04635816802179016
250 0.08036849128672874
275 0.1057413825572231
300 0.2398966278472554
325 0.21293230421806064
350 0.19607656957356612
375 0.9110994879973344
400 0.2534662867102575
425 0.04241440047590894
450 0.1083049464453486
475 0.7917934514817366
500 0.1948161501501462
525 1.1042811808840751
550 0.02148734249962303
575 0.23404759265406017
600 0.4529306723800166
625 0.44599797678358066
650 0.7844706080460386
675 0.09593812416224978
700 0.27000603860508343
725 0.17608893581523377
750 0.05199757947918482
775 0.12140623372271842
800 0.10272810921238895
825 0.11513810816314729
850 0.4447967208274586
875 0.032212817838655816
900 0.02898657545794814
925 0.036251844898376406
950 0.17510187583504055
975 0.03010532335792304


KeyboardInterrupt: 

In [453]:
X = new_sol.ncgm.xgrid
X

array([  10.,   20.,   30.,   40.,   50.,   60.,   70.,   80.,   90.,
        100.,  110.,  120.,  130.,  140.,  150.,  160.,  170.,  180.,
        190.,  200.,  210.,  220.,  230.,  240.,  250.,  260.,  270.,
        280.,  290.,  300.,  310.,  320.,  330.,  340.,  350.,  360.,
        370.,  380.,  390.,  400.,  410.,  420.,  430.,  440.,  450.,
        460.,  470.,  480.,  490.,  500.,  510.,  520.,  530.,  540.,
        550.,  560.,  570.,  580.,  590.,  600.,  610.,  620.,  630.,
        640.,  650.,  660.,  670.,  680.,  690.,  700.,  710.,  720.,
        730.,  740.,  750.,  760.,  770.,  780.,  790.,  800.,  810.,
        820.,  830.,  840.,  850.,  860.,  870.,  880.,  890.,  900.,
        910.,  920.,  930.,  940.,  950.,  960.,  970.,  980.,  990.,
       1000.])

In [454]:
idx = np.where(X == 270)[0][0]
VF[idx]

30.219417251277438

In [455]:
new_sol.simulate(274,1000,s=1)

(57.75810698575455, 0.0, 0, 23)

In [456]:
VF = new_sol.VF
VF

array([ 3.25827747,  5.07297805,  6.48184022,  7.67903694,  8.73824426,
        9.69735504, 10.57908614, 11.39840583, 12.16583449, 12.88912275,
       13.57418935, 14.22568397, 14.8473439 , 15.44223056, 16.01289188,
       16.56147737, 17.08982176, 17.59950714, 18.09190999, 18.56823747,
       19.02955579, 19.47681263, 19.9108552 , 20.33244484, 20.742269  ,
       21.14095108, 21.52905868, 21.90711052, 22.27558222, 22.63491133,
       22.9855015 , 23.32772615, 23.66193164, 23.98843998, 24.30755124,
       24.61954562, 24.92468529, 25.22321605, 25.51536871, 25.80136042,
       26.08139577, 26.35566784, 26.62435912, 26.88764228, 27.14568096,
       27.39863044, 27.64663822, 27.88984458, 28.12838311, 28.3623811 ,
       28.59196005, 28.81723595, 29.03831974, 29.25531752, 29.46833094,
       29.67745742, 29.88279041, 30.08441962, 30.28243128, 30.47690826,
       30.66793032, 30.85557427, 31.0399141 , 31.22102116, 31.39896428,
       31.57380993, 31.74562231, 31.91446347, 32.08039343, 32.24

In [ ]:
new_sol.build_VF()

In [ ]:
CP = new_sol.CP
CP

In [ ]:
new_sol.build_CP()

In [ ]:
WP = new_sol.WP
WP

In [ ]:
new_sol.build_WP()

In [ ]:
CP/WP

In [ ]:
xnew = np.arange(1,2e3,100)
vf_new = new_sol.build_VF(xnew)

plt.plot(X, VF, 'o', xnew, vf_new)

In [ ]:
cp_new = new_sol.build_CP(xnew)

plt.plot(X, CP, 'o', xnew, cp_new)

In [ ]:
wp_new = new_sol.build_WP(xnew)

plt.plot(X, WP, 'o', xnew, wp_new)

In [ ]:
(cp_new/wp_new)[1000:1050]